<a href="https://colab.research.google.com/github/tnc-br/ddf-insights-analytics/blob/main/fraud_detection_enhancement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Meta Data and Parameters**

In [35]:
lat = -9
lon = -40
oxygen_measurements = [32,33,34]
nitrogen_measurements = [22,23,24]
carbon_measurements = [-20,-21,-23]

Requirements

In [3]:
#Install requirements


Libraries

In [52]:
#Import Libraries
import ee
import os
import numpy as np
import scipy


Authentication and Authorization

In [2]:
#Authenticate
if os.path.isfile('key.json') :

    # connection to the service account
    service_account = 'fraud-detection-pipeline-sa@kazob-370920.iam.gserviceaccount.com'
    credentials = ee.ServiceAccountCredentials(service_account, 'key.json')
    ee.Initialize(credentials)
    print('initialized with key')

# if in local env use the local user credential
else:
    ee.Initialize()

initialized with key


Global Variable definition

In [ ]:
isoscapes_ee_path = 'users/kazob/isoscapes'

Datasets

In [ ]:
# brazil_bounds = ee.FeatureCollection('FAO/GAUL/2015/level1').filter(ee.Filter.eq('ADM0_NAME', 'Brazil'))
# Map.addLayer(brazil_bounds, {}, 'Brazil')
# Map.centerObject(brazil_bounds,4)

Code

fetching isoscapes

In [13]:
parent_asset = ee.data.getAsset(isoscapes_ee_path)
parent_id = parent_asset['name']
parent_type = parent_asset['type']
asset_list = []
child_assets = ee.data.listAssets({'parent': parent_id})['assets']
for child_asset in child_assets:
    child_id = child_asset['name']
    child_type = child_asset['type']
    if child_type in ['FOLDER','IMAGE_COLLECTION']:
        # Recursively call the function to get child assets
        asset_list.extend(get_asset_list(child_id))
    else:
        asset_list.append(child_id)
asset_list

['projects/earthengine-legacy/assets/users/kazob/isoscapes/xgb_means_oxygen_isoscape',
 'projects/earthengine-legacy/assets/users/kazob/isoscapes/xgb_variances_oxygen_isoscape']

In [26]:
for asset in asset_list:
  if ('oxygen' in asset) and ('mean' in asset):
    oxygen_means_iso = ee.Image(asset)
  elif 'oxygen' in asset and 'variance' in asset:
    oxygen_variances_iso = ee.Image(asset)
  elif 'd13C' in asset:
    carbon_variances_iso = ee.Image(asset).select('B0')
    carbon_means_iso = ee.Image(asset).select('B1')


POI definition

In [56]:
poi = ee.Geometry.Point(lon,lat)

***Oxygen***

Isoscape

In [ ]:
isoscape_mean = oxygen_means_iso.reduceRegion(
  reducer = ee.Reducer.mean(),
  geometry = poi,
  scale = 30).getInfo().get('b1');
isoscape_var = oxygen_variances_iso.reduceRegion(
  reducer = ee.Reducer.mean(),
  geometry = poi,
  scale = 30).getInfo().get('b1');
if isoscape_var is not None:
  isoscape_std = np.sqrt(isoscape_var)
else:
  isoscape_std = 0

Isotope calculation

In [57]:
isotope_mean = np.mean(oxygen_measurements)
isotope_std = np.std(oxygen_measurements)
print(isotope_mean,isotope_std, isoscape_mean, isoscape_std)



33.0 0.816496580927726 43.059565734863284 0.6551787351470115


ttest and p-value

In [68]:
_, p_value_oxygen = scipy.stats.ttest_ind_from_stats(
            isotope_mean, isotope_std, len(oxygen_measurements), isoscape_mean, isoscape_std, 30, equal_var=False, alternative="two-sided")

**Carbon**

Isoscape

In [60]:
#delete
carbon_means_iso = oxygen_means_iso
carbon_variances_iso = oxygen_variances_iso
#delete

isoscape_mean = carbon_means_iso.reduceRegion(
  reducer = ee.Reducer.mean(),
  geometry = poi,
  scale = 30).getInfo().get('b1');
isoscape_var = carbon_variances_iso.reduceRegion(
  reducer = ee.Reducer.mean(),
  geometry = poi,
  scale = 30).getInfo().get('b1');
if isoscape_var is not None:
  isoscape_std = np.sqrt(isoscape_var)
else:
  isoscape_std = 0

Isotope calculation

In [61]:
isotope_mean = np.mean(carbon_measurements)
isotope_std = np.std(carbon_measurements)
print(isotope_mean,isotope_std, isoscape_mean, isoscape_std)



-21.333333333333332 1.247219128924647 43.059565734863284 0.6551787351470115


ttest and p-value

In [67]:
_, p_value_carbon = scipy.stats.ttest_ind_from_stats(
            isotope_mean, isotope_std, len(carbon_measurements), isoscape_mean, isoscape_std, 30, equal_var=False, alternative="two-sided")

**Nitrogen**

Isoscape

In [63]:
#delete
nitrogen_means_iso = oxygen_means_iso
nitrogen_variances_iso = oxygen_variances_iso
#delete

isoscape_mean = nitrogen_means_iso.reduceRegion(
  reducer = ee.Reducer.mean(),
  geometry = poi,
  scale = 30).getInfo().get('b1');
isoscape_var = nitrogen_variances_iso.reduceRegion(
  reducer = ee.Reducer.mean(),
  geometry = poi,
  scale = 30).getInfo().get('b1');
if isoscape_var is not None:
  isoscape_std = np.sqrt(isoscape_var)
else:
  isoscape_std = 0

Isotope calculation

In [64]:
isotope_mean = np.mean(carbon_measurements)
isotope_std = np.std(carbon_measurements)
print(isotope_mean,isotope_std, isoscape_mean, isoscape_std)



-21.333333333333332 1.247219128924647 43.059565734863284 0.6551787351470115


ttest and p-value

In [65]:
_, p_value_nitrogen = scipy.stats.ttest_ind_from_stats(
            isotope_mean, isotope_std, len(carbon_measurements), isoscape_mean, isoscape_std, 30, equal_var=False, alternative="two-sided")

Origin Verification

In [69]:
origin_validity = p_value_oxygen * p_value_carbon * p_value_nitrogen